In [1]:
# Kernel should be set to Python 2 to accomodate Multitracker

import multi_tracker_analysis as mta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from moviepy.editor import *
import glob, math, os
from __future__ import division

%load_ext watermark
%watermark -a "Eleanor Lutz" -v -r -w -m -iv -i

1.5.1
recommended version: 1.1.1 or greater
numpy       1.13.1
pandas      0.19.2
Eleanor Lutz 2018-04-19T10:44:13

CPython 2.7.12
IPython 5.4.1

compiler   : GCC 5.4.0 20160609
system     : Linux
release    : 4.13.0-38-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit
Git repo   : https://github.com/eleanorlutz/multivideo_multitracker
watermark 1.6.0


In [2]:
'''
Translate Multitracker hdf5 files into Pandas dataframes and save to CSV.
This preprocessing step needs to be done before any of the other code
to analyze speed, velocity, etc. 

The script will not overwrite files that have already been created, so it can be 
run multiple times until the output is a one-line: 
"Script finished running!"

If files are being processed, Multitracker will output various lines of text
that are propagated into the python output. 
'''

fnames = glob.glob('/home/eleanor/Documents/gcamp_analysis_files_finished/*/data/')
df_names = pd.read_csv('/home/eleanor/Documents/gitrepos/mosquito_behavior_analysis/static_data_all_GCAMP.csv')

for n in range(0, len(fnames)):
    fname = fnames[n]
    name = fname[54:-6]
    
    videoname = '/home/eleanor/Documents/gcamp_videos/' + name + '.avi'
    savename = '/home/eleanor/Documents/gitrepos/mosquito_behavior_analysis/gcamp_video_csvs/'+ name +'.csv'
    
    # Do not overwrite video files that have already been made and reviewed
    if not os.path.isfile(savename):
        df, config = mta.read_hdf5_file_to_pandas.load_and_preprocess_data(fname)

        video_clip = VideoFileClip(videoname)
        frame = np.array(video_clip.get_frame(0).astype(float))
        video_width, video_height = len(frame[0]), len(frame)
        df['pixel_width'] = video_width
        df['pixel_height'] = video_height
        
        # Mark frames detected by Multitracker to differentiate from manually entered frames.
        df['manual_tracker_fix'] = False
        
        # remove default columns added by multitracker that are not relevant for mosquito larvae. 
        del df['angle'], df['area'], df['time_epoch'], df['time_epoch_nsecs'], df['time_epoch_secs'], \
        df['speed'], df['velocity_x'], df['velocity_y']

        df.to_csv(savename, index=False)
        del video_clip.reader, video_clip
        
print('Script finished running!')

Script finished running!


In [3]:
'''
Multitracker will not register objects until they begin to move.
In some videos, animals do not start to move until several seconds into the experiment. 

Each of these videos are manually inspected to confirm lack of movement in the initial frames.
(This involves just opening each video in ImageJ Fiji and fast forwarding
to confirm that the animal is not moving for the missing frames).

This script prints out a list of all csv files with missing frames. 
'''

# Print all files with missing frames for manual inspection. 
fnames = sorted(glob.glob('./gcamp_video_csvs/*.csv'))[::-1] # reversed list
error_fnames = []
count = 0

for name in fnames:
    df = pd.read_csv(name)
    missed = 1799 - len(df)
    if missed > 2: # first frame is always missing because of background subtraction. 
        error_fnames.append(name[19:])
        print name[19:], ':', 1799-len(df), 'frames missing'
        count += 1
        
print '---'
print count, 'files have errors, out of', len(fnames), 'total files'

---
0 files have errors, out of 0 total files


In [4]:
'''
Run this script once the video has been manually checked to see if it has a long period of 
no motion at the beginning. 

This script reads in the file list of manually inspected frames, and propagates
the tracker's position when the larva was first detected to the beginning of the movie. 
All manually corrected frames are marked with "manual" in the tracker ID column.

If the file has already been rewritten the script will not make any further changes. 
This is because the file is additionally checked in each loop for missing beginning frames, 
and only the missing data is added. 
'''

checked = pd.read_csv('./manually_checked_beginning_pause.csv')
checked_fnames = checked['filename'].tolist()
count = 0

for name in checked_fnames: 
    name_readable = "/home/eleanor/Documents/gitrepos/mosquito_behavior_analysis//gcamp_video_csvs/"+name+".csv"
    df = pd.read_csv(name_readable)

    frames = df['frames'].tolist()
    fmin = min(frames)
    fmin_row = df[df['frames'] == fmin]
    assert len(fmin_row) == 1
    
    if fmin > 2:
        for n in range(2, fmin):
            fmin_copy = fmin_row.copy()
            fmin_copy.ix[0, "objid"] = "manual"
            fmin_copy.ix[0, "frames"] = n
            df = df.append(fmin_copy)
            
        df = df.sort_values(by="frames")
        df.to_csv(name_readable, index=False)
        count += 1
    
print('Finished analyzing '+str(len(checked_fnames))+' files!')
print('Rewrote '+str(count)+' files!')

Finished analyzing 140 files!
Rewrote 0 files!
